In [2]:
import gym

from stable_baselines3 import PPO


2024-03-25 15:57:25.995049: I tensorflow/stream_executor/platform/default/dso_loader.cc:50] Successfully opened dynamic library libcudart.so.12


In [3]:
env=gym.make('Pendulum-v1')

In [4]:
model=PPO('MlpPolicy',env,verbose=0)
model.learn(1000,progress_bar=1)

/home/data/envs/IL/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Output()

In [5]:
model.save('model.zip')

In [13]:
model=PPO('model.zip')

TypeError: __init__() missing 1 required positional argument: 'env'

In [7]:
from stable_baselines3.common.evaluation import evaluate_policy

In [9]:
reward,std=evaluate_policy(model,env,1000)

/home/data/envs/IL/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/home/data/envs/IL/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [11]:
std

307.663940481476